In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import numpy as np
import time
import gc

In [ ]:
vertices = pd.read_csv(r'\data\vertices.csv') ## вершины ребер, то есть юр.лица
edges = pd.read_csv(r'\data\edges.csv') ## ребра графа, то есть транзакции между юр.лицами

####  Сгруппируем информацию о транзациях по разным уровням

In [ ]:
merged = pd.merge(edges, vertices, left_on = 'id_1', right_on = 'id')\
    .rename(columns = {'main_okved' : 'main_okved_1',
                       'region_code' : 'region_code_1',
                       'company_type' : 'company_type_1'})

edges = merged.merge(vertices, left_on = 'id_2', right_on = 'id')\
    .rename(columns = {'main_okved' : 'main_okved_2',
                       'region_code' : 'region_code_2',
                       'company_type' : 'company_type_2'})

edges = edges.drop(['id_x', 'id_y'], axis = 'columns')

In [ ]:
grouped1 = edges.groupby('id_1', axis = 0)
grouped2 = edges.groupby('id_2', axis = 0)

In [ ]:
id1_keys = grouped1.groups.keys()
id2_keys = grouped2.groups.keys()

In [ ]:
%%time
all_firms = pd.Series(np.append(edges['id_1'].values, edges['id_2'].values)).sort_values().unique()
vertices_features = pd.DataFrame(columns = ['value_mean', 'value_std', 'value_sum', 'n_transactions_mean',
       'n_transactions_std', 'n_transactions_sum'])

def feature_extraction(x):
    global vertices_features

    try:
        df1 = grouped1.get_group(x)
    except:
        'Ba-dum-tss'
    
    try:
        df2 = grouped2.get_group(x)
    except:
        'Ba-dum-tss'
    
    try:
        df = df1.append(df2, ignore_index = True)
    except:
        try:
            df = df1
        except:
            df = df2
    
    vertices_features = vertices_features.append(pd.DataFrame({'id' : [x],
                          'value_mean' :[df['value'].mean()],
                          'value_std' :[df['value'].std()],
                          'value_sum' :[df['value'].sum()],
                          'n_transactions_mean' : [df['n_transactions'].mean()],
                          'n_transactions_std' : [df['n_transactions'].std()],
                          'n_transactions_sum' : [df['n_transactions'].sum()]}))

In [ ]:
map(feature_extraction, all_firms[1018395:])

In [ ]:
vertices.merge(vertices_features.drop_duplicates()).to_csv('vertices_features.csv', index = False)